In [1]:
import glob
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go
from parser import SymbTextParser

In [2]:
GOLDEN_RATIO = (1.0 + np.sqrt(5)) / 2.0
EPSILON = 0.002

In [14]:
GOLDEN_RATIO

1.618033988749895

In [3]:
def get_duration_dict(file_path, parser: SymbTextParser, as_pitch_classes=True):
    pitches, durations = parser.get_pitch_classes_with_durations(
        [file_path], as_pitch_classes=as_pitch_classes)
    dur_dict = {}
    for pitch, duration in zip(pitches[0], durations[0]):
        if pitch not in dur_dict:
            dur_dict[pitch] = duration
        else:
            dur_dict[pitch] += duration

    return dur_dict

In [4]:
def get_ratio_matrix(dur_dict: dict):
    ratio_mat = np.zeros((len(dur_dict), len(dur_dict)))
    i = 0
    pitches_i = []
    pitches_j = []
    for pitch_i, dur_i in dur_dict.items():
        pitches_i.append(pitch_i)
        pitches_j.append(pitch_i)
        j = 0
        for pitch_j, dur_j in dur_dict.items():
            ratio_mat[i][j] = dur_i / dur_j
            j += 1
        i += 1
    return ratio_mat, pitches_i, pitches_j

In [5]:
def plot_heatmap(ratio_matrix, pitches_i, pitches_j, title, save_pdf=False):
    ratio_matrix[ratio_matrix > GOLDEN_RATIO + 2*EPSILON] = None
    ratio_matrix[ratio_matrix < GOLDEN_RATIO - 2*EPSILON] = None
    fig = go.Figure(data=go.Heatmap(
        z=ratio_matrix,
        x=pitches_i,
        y=pitches_j,
        reversescale=True
    ))

    fig.update_traces(text=ratio_matrix, texttemplate="%{text:.3f}")
    fig.update_xaxes(side="top")
    fig.update_yaxes(autorange="reversed")
    fig.update_layout(title=title,
                      title_x=0.5,
                      margin=dict(t=50, r=10, b=10, l=10),
                      showlegend=False,
                      width=800,
                      height=800)
    if save_pdf:
        pio.write_image(fig,
                        f"figures/{title.replace(" ", "-")}.png",
                        scale=6, width=800, height=800)
    fig.show()

In [6]:
def plot_heatmap_by_path(file_path, title, save_pdf=False, as_pitch_classes=True):
    parser = SymbTextParser()
    dur_dict = get_duration_dict(
        file_path, parser, as_pitch_classes=as_pitch_classes)
    ratio_matrix, pitches_i, pitches_j = get_ratio_matrix(dur_dict)
    plot_heatmap(ratio_matrix, pitches_i, pitches_j, title, save_pdf=save_pdf)

In [8]:
parser = SymbTextParser()
file_list = glob.glob("data/txt/*.txt")
for f in file_list:
    dur_dict = get_duration_dict(f, parser, as_pitch_classes=False)
    ratio_matrix, pitches_i, pitches_j = get_ratio_matrix(dur_dict)
    difference_array = np.absolute(ratio_matrix-GOLDEN_RATIO)

    if len(difference_array[difference_array < EPSILON]) < 2:
        continue

    print(f)
    song_title = f.replace("data/txt/", "").replace(".txt", "")\
        .replace("--", " ").replace("_", " ")
    plot_heatmap_by_path(f, song_title, save_pdf=True, as_pitch_classes=False)

data/txt/acemasiran--beste--devrikebir--ber_kusa-yi--basmaci_abdi_efendi.txt


data/txt/beyati--sarki--agiraksak--bir_katre--sevki_bey.txt


data/txt/hicazkar--sarki--devrihindi--gonlumu_ducar--sevki_bey.txt


data/txt/acemasiran--sarki--agiraksak--goncasindan_gulsenin--emin_ongan.txt


data/txt/huseyni--divan--sofyan----.txt


...

data/txt/acemasiran--sarki--duyek--gonul_seni--sadettin_kaynak.txt


data/txt/kurdilihicazkar--sazsemaisi--aksaksemai--sonbahar--coskun_acikgoz.txt


data/txt/nihavent--sarki--semai--bir_nese--nuri_halil_poyraz.txt


data/txt/kurdilihicazkar--sarki--musemmen--gecti_omrun--sadi_hosses.txt


data/txt/hicaz--mandra--devrituran_ii----lavtaci_andon.txt


In [10]:
fp = "data/txt/acemasiran--sarki--duyek--gonul_seni--sadettin_kaynak.txt"
dur_dict = get_duration_dict(fp, parser, as_pitch_classes=False)
ratio_matrix, pitches_i, pitches_j = get_ratio_matrix(dur_dict)

In [13]:
float(dur_dict["D5"]/dur_dict["A5"])

1.619047619047619